## Outreach for each post

I successivi blocchi di codice consentono di fare un confronto tra la len dell'interazione e la size dell'outreach di una pagina misurata come Lurkers in quel momento.

In [ ]:
import pandas as pd
import os
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt

/home/jacoponudo/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [ ]:
posts = pd.read_csv('/home/jacoponudo/Documents/Size_effects/DATA/sample_posts.csv',usecols=['created_time', 'page_id', 'post_id'],encoding='ISO-8859-1')
likes = pd.read_csv('/home/jacoponudo/Documents/Size_effects/DATA/sample_likes.csv', usecols=['from_id', 'post_id'], encoding='ISO-8859-1')
np.mean(posts['post_id'].isin(likes['post_id'])) # Solo del 66% dei post abbiamo i like, poichè questi sono stati ignorati per le pagine troppo grandi. Vedi EXP/check_likes_consistency.ipynb

np.float64(0.6627619477450383)

Estraiamo per ciascuna pagine la serie storica dei Lurkers, ovvero il numero di utenti univoci che ha lasciato like a un qualsiasi post della pagina.

In [ ]:
folder_path = '/home/jacoponudo/Documents/Size_effects/DATA/'
dataframes = []

# Itera su ogni ID di pagina unico
for page_id in tqdm(posts['page_id'].unique()):
    page_posts = posts[posts['page_id'] == page_id]
    page_likes = likes[likes['post_id'].isin(page_posts['post_id'].unique())]

    # Unisci i dati dei post con i dati dei like
    complete_page_likes = pd.merge(page_posts, page_likes, on='post_id', how='right')
    complete_page_likes['created_time'] = pd.to_datetime(complete_page_likes['created_time'])
    complete_page_likes.set_index('created_time', inplace=True)

    # Calcola il numero di utenti unici settimanali
    weekly_unique_users = complete_page_likes['from_id'].resample('W').nunique()
    monthly_moving_avg = weekly_unique_users.rolling(window=8).mean()
    
    # Crea il DataFrame per la media mobile mensile
    monthly_avg_df = pd.DataFrame({
        'Date': monthly_moving_avg.index,
        'Monthly Moving Average': monthly_moving_avg.values
    }).dropna() 
    
    # Aggiungi l'ID della pagina
    monthly_avg_df.insert(2, 'page_id', page_id)
    dataframes.append(monthly_avg_df)

# Concatenazione di tutti i DataFrame in un unico DataFrame outreach
outreachs = pd.concat(dataframes, ignore_index=True)

# Salva il file completo di outreach
output_file = f'{folder_path}/sample_outreachs.csv'
outreachs.to_csv(output_file, index=False)

print(f"File salvato in: {output_file}")

100%|██████████| 117/117 [08:48<00:00,  4.51s/it]

File salvato in: /home/jacoponudo/Documents/Size_effects/DATA//sample_outreachs.csv


Puliamo la ram da tutti i file in memoria.

In [ ]:
import gc
def clear_all():
    for name in dir():
        if not name.startswith('_'):
            del globals()[name]
    gc.collect()

# Pulisci la RAM da tutte le variabili
clear_all()

Per ogni interazione, calcolo quanto dura (interaction len) e dove avviene (page_id), e qual'era il valore di Luckers sulla pagina su cui avviene in quel momento.

In [ ]:
comments = pd.read_csv('/home/jacoponudo/Documents/Size_effects/DATA/sample_comments.csv',encoding='ISO-8859-1',usecols=['from_id', 'post_id'])
interactions = comments.groupby(['from_id', 'post_id']).size().reset_index(name='interaction_len')
del comments
gc.collect()

0

Interpolazione dell'outreach di ogni post.

In [ ]:
posts = pd.read_csv('/home/jacoponudo/Documents/Size_effects/DATA/sample_posts.csv',usecols=['created_time', 'page_id', 'post_id'],encoding='ISO-8859-1')
outreach_values = []

for idx, post in tqdm(posts.iterrows(), total=posts.shape[0]):
    page_id = post['page_id']
    outreach = outreachs[outreachs['page_id'] == int(page_id)].copy()
    target_date = pd.to_datetime(post['created_time'])
    previous_date_row = outreach[outreach['Date'] < target_date]
    if not previous_date_row.empty:
        monthly_moving_avg = previous_date_row.loc[previous_date_row['Date'].idxmax()]['Monthly Moving Average']
    else:
        monthly_moving_avg = None
    outreach_values.append(monthly_moving_avg)
posts['outreach'] = outreach_values

interactions_selected = interactions[['interaction_len', 'post_id']]
interactions_selected['alpha'] = (interactions['interaction_len'] < 2)
posts_selected = posts[['post_id', 'outreach']]
merged_df = pd.merge(interactions_selected, posts_selected, on='post_id', how='inner')
merged_df.to_csv('/home/jacoponudo/Documents/Size_effects/DATA/facebook_outreach_vs_interaction.csv')

100%|██████████| 1482309/1482309 [24:08<00:00, 1023.59it/s]
